# Residual Gated Graph ConvNets
### Xavier Bresson, Jan. 15 2018

In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import pdb 
import time
import numpy as np
import pickle

if torch.cuda.is_available():
    print('cuda available')
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    #torch.cuda.manual_seed(1)
else:
    print('cuda not available')
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    #torch.manual_seed(1)
    
# import files in folder util
import core.block as block
import core.graph_generator as g
from core.GraphConvNet import GraphConvNet

from sklearn.metrics import confusion_matrix

cuda not available
cuda not available


In [3]:

#################
# select task and task parameters
#################


# subgraph matching
if 2==1:
    task_parameters = {}
    task_parameters['flag_task'] = 'matching'
    task_parameters['nb_communities'] = 10
    task_parameters['nb_clusters_target'] = 2
    task_parameters['Voc'] = 3
    task_parameters['size_min'] = 15
    task_parameters['size_max'] = 25
    task_parameters['size_subgraph'] = 20
    task_parameters['p'] = 0.5
    task_parameters['q'] = 0.1
    task_parameters['W0'] = block.random_graph(task_parameters['size_subgraph'],task_parameters['p'])
    task_parameters['u0'] = np.random.randint(task_parameters['Voc'],size=task_parameters['size_subgraph'])

    
# semi-supervised clustering
if 1==1:
    task_parameters = {}
    task_parameters['flag_task'] = 'clustering'
    task_parameters['nb_communities'] = 10
    task_parameters['nb_clusters_target'] = task_parameters['nb_communities']
    task_parameters['Voc'] = task_parameters['nb_communities'] + 1
    task_parameters['size_min'] = 5
    task_parameters['size_max'] = 25
    task_parameters['p'] = 0.5
    task_parameters['q'] = 0.1

In [4]:

#################
# network and optimization parameters
#################

# network parameters
net_parameters = {}
net_parameters['Voc'] = task_parameters['Voc']
net_parameters['D'] = 50
net_parameters['nb_clusters_target'] = task_parameters['nb_clusters_target']
net_parameters['H'] = 50
net_parameters['L'] = 10
#print(net_parameters)


# optimization parameters
opt_parameters = {}
opt_parameters['learning_rate'] = 0.00075   # ADAM
opt_parameters['max_iters'] = 5000   
opt_parameters['batch_iters'] = 100
if 1==1: # fast debugging
    opt_parameters['max_iters'] = 101 
    opt_parameters['batch_iters'] = 10
opt_parameters['decay_rate'] = 1.25   
#print(opt_parameters)


In [5]:
 
#########################
# Graph convnet function
#########################
def our_graph_convnets(task_parameters,net_parameters,opt_parameters):


    # Delete existing network if exists
    try:
        del net
        print('Delete existing network\n')
    except NameError:
        print('No existing network to delete\n')


    # instantiate
    net = GraphConvNet(net_parameters, task_parameters)
    if torch.cuda.is_available():
        net.cuda()
    print(net)
    
    
    # number of network parameters
    nb_param = 0
    for param in net.parameters():
        nb_param += np.prod(list(param.data.size()))
    print('nb_param=',nb_param,' L=',net_parameters['L'])
    

    # task parameters
    flag_task = task_parameters['flag_task']
    # network parameters
    Voc = net_parameters['Voc']
    D = net_parameters['D']
    nb_clusters_target = net_parameters['nb_clusters_target']
    H = net_parameters['H']
    L = net_parameters['L']
    # optimization parameters
    learning_rate = opt_parameters['learning_rate']
    max_iters = opt_parameters['max_iters']
    batch_iters = opt_parameters['batch_iters']
    decay_rate = opt_parameters['decay_rate']
    
    
    # Optimizer
    global_lr = learning_rate
    global_step = 0
    lr = learning_rate
    optimizer = net.update(lr) 

    
    #############
    # loop over epochs
    #############
    t_start = time.time()
    t_start_total = time.time()
    average_loss_old = 1e10
    running_loss = 0.0
    running_total = 0
    running_conf_mat = 0
    running_accuracy = 0
    tab_results = []
    for iteration in range(1*max_iters):  # loop over the dataset multiple times

        # generate one train graph
        if flag_task=='matching': # subgraph matching
            train_x = g.variable_size_graph(task_parameters)
        elif flag_task=='clustering': # semi supervised clustering
            train_x = g.graph_semi_super_clu(task_parameters)
        train_y = train_x.target
        train_y = Variable( torch.LongTensor(train_y).type(dtypeLong) , requires_grad=False) 

        # forward, loss
        y = net.forward(train_x)
        # compute loss weigth
        labels = train_y.data.cpu().numpy()
        V = labels.shape[0]
        nb_classes = len(np.unique(labels)) 
        cluster_sizes = np.zeros(nb_classes)
        for r in range(nb_classes):
            cluster = np.where(labels==r)[0]
            cluster_sizes[r] = len(cluster)    
        weight = torch.zeros(nb_classes)
        for r in range(nb_classes):
            sumj = 0
            for j in range(nb_classes):
                if j!=r:
                    sumj += cluster_sizes[j]
            weight[r] = sumj/ V 
        loss = net.loss(y,train_y,weight)
        loss_train = loss.data[0]
        running_loss += loss_train
        running_total += 1

        # confusion matrix
        S = train_y.data.cpu().numpy()
        C = np.argmax( torch.nn.Softmax(dim=0)(y).data.cpu().numpy() , axis=1)
        CM = confusion_matrix(S,C).astype(np.float32)
        nb_classes = CM.shape[0]
        train_y = train_y.data.cpu().numpy()
        for r in range(nb_classes):
            cluster = np.where(train_y==r)[0]
            CM[r,:] /= cluster.shape[0]
        running_conf_mat += CM
        running_accuracy += np.sum(np.diag(CM))/ nb_classes

        # backward, update
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # learning rate, print results
        if not iteration%batch_iters:

            # time
            t_stop = time.time() - t_start
            t_start = time.time()

            # confusion matrix
            average_conf_mat = running_conf_mat/ running_total
            running_conf_mat = 0

            # accuracy
            average_accuracy = running_accuracy/ running_total
            running_accuracy = 0

            # update learning rate 
            average_loss = running_loss/ running_total
            if average_loss > 0.99* average_loss_old:
                lr /= decay_rate
            average_loss_old = average_loss
            optimizer = net.update_learning_rate(optimizer, lr)
            running_loss = 0.0
            running_total = 0

            # save intermediate results
            tab_results.append([iteration,average_loss,100* average_accuracy,time.time()-t_start_total])

            # print results
            if 1==1:
                print('\niteration= %d, loss(%diter)= %.3f, lr= %.8f, time(%diter)= %.2f' % 
                      (iteration, batch_iters, average_loss, lr, batch_iters, t_stop))
                #print('Confusion matrix= \n', 100* average_conf_mat)
                print('accuracy= %.3f' % (100* average_accuracy))
    
    return net, optimizer


#run it
net, optimizer = our_graph_convnets(task_parameters,net_parameters,opt_parameters)


No existing network to delete


nb of hidden layers= 10
dim of layers (w/ embed dim)= [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]


GraphConvNet(
  (encoder): Embedding(11, 50)
  (gnn_cells): ModuleList(
    (0): GraphConvNetcell(
      (Ui1): Linear(in_features=50, out_features=50, bias=False)
      (Uj1): Linear(in_features=50, out_features=50, bias=False)
      (Vi1): Linear(in_features=50, out_features=50, bias=False)
      (Vj1): Linear(in_features=50, out_features=50, bias=False)
      (Ui2): Linear(in_features=50, out_features=50, bias=False)
      (Uj2): Linear(in_features=50, out_features=50, bias=False)
      (Vi2): Linear(in_features=50, out_features=50, bias=False)
      (Vj2): Linear(in_features=50, out_features=50, bias=False)
      (bn1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (R): Linear(in_features=50, out_features=50, bias=False)

/Users/signapoop/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/signapoop/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:91: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



iteration= 0, loss(10iter)= 2.384, lr= 0.00075000, time(10iter)= 0.40
accuracy= 8.808

iteration= 10, loss(10iter)= 2.301, lr= 0.00075000, time(10iter)= 1.93
accuracy= 12.999

iteration= 20, loss(10iter)= 2.185, lr= 0.00075000, time(10iter)= 2.13
accuracy= 24.207

iteration= 30, loss(10iter)= 2.110, lr= 0.00075000, time(10iter)= 1.78
accuracy= 28.748

iteration= 40, loss(10iter)= 2.056, lr= 0.00075000, time(10iter)= 1.84
accuracy= 29.186

iteration= 50, loss(10iter)= 1.970, lr= 0.00075000, time(10iter)= 1.74
accuracy= 37.640

iteration= 60, loss(10iter)= 1.839, lr= 0.00075000, time(10iter)= 1.81
accuracy= 41.984

iteration= 70, loss(10iter)= 1.733, lr= 0.00075000, time(10iter)= 2.00
accuracy= 45.211

iteration= 80, loss(10iter)= 1.604, lr= 0.00075000, time(10iter)= 1.56
accuracy= 53.041

iteration= 90, loss(10iter)= 1.521, lr= 0.00075000, time(10iter)= 1.90
accuracy= 53.184

iteration= 100, loss(10iter)= 1.364, lr= 0.00075000, time(10iter)= 1.85
accuracy= 57.399


In [ ]:
def save_checkpoint(state, filename='results/checkpoint2.pth.tar'):
    torch.save(state, filename)
    
save_checkpoint({
    'state_dict': net.state_dict(),
    'optimizer' : optimizer.state_dict(),
})

In [6]:
test_x = g.graph_semi_super_clu(task_parameters)
test_y = test_x.target
test_y = Variable( torch.LongTensor(test_y).type(dtypeLong) , requires_grad=False) 

# forward, loss
y = net.forward(test_x)
labels = test_y.data.cpu().numpy()

/Users/signapoop/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [7]:
test_x.target

tensor([7, 7, 7, 6, 7, 5, 7, 9, 1, 7, 7, 5, 5, 1, 2, 2, 0, 1, 7, 6, 2, 9, 6, 2,
        6, 8, 5, 9, 7, 8, 7, 5, 1, 3, 1, 5, 6, 8, 6, 2, 7, 3, 7, 5, 2, 8, 6, 2,
        0, 6, 1, 8, 8, 0, 1, 3, 4, 3, 7, 1, 7, 1, 6, 7, 8, 1, 2, 6, 2, 3, 3, 9,
        4, 1, 5, 5, 1, 4, 8, 7, 8, 8, 5, 6, 2, 5, 4, 6, 1, 5, 4, 5, 2, 7, 6, 5,
        7, 8, 2, 2, 1, 1, 8, 7, 5, 2, 8, 3, 8, 8, 7, 5, 2, 7, 0, 5, 3, 2, 0, 5,
        5, 2, 0, 6, 0, 1, 8, 1, 2, 8, 4, 1, 7, 2, 9, 2, 1, 6, 5, 2])

In [8]:
C = np.argmax( torch.nn.Softmax(dim=0)(y).data.cpu().numpy() , axis=1)

In [9]:
C

array([7, 1, 7, 9, 7, 5, 7, 8, 1, 7, 7, 0, 5, 1, 2, 3, 3, 1, 7, 0, 2, 9,
       0, 3, 6, 8, 0, 9, 4, 8, 7, 0, 1, 6, 8, 5, 0, 6, 0, 3, 7, 3, 7, 5,
       2, 0, 0, 9, 3, 9, 6, 0, 0, 0, 1, 5, 8, 8, 8, 6, 7, 6, 9, 7, 0, 4,
       9, 5, 9, 3, 3, 5, 4, 1, 5, 5, 1, 4, 3, 7, 0, 8, 5, 0, 2, 5, 7, 9,
       1, 5, 8, 8, 3, 7, 0, 5, 7, 8, 9, 9, 1, 4, 8, 7, 5, 9, 0, 3, 8, 8,
       7, 8, 2, 7, 4, 9, 3, 9, 6, 5, 5, 2, 4, 0, 0, 1, 4, 5, 9, 4, 4, 1,
       8, 9, 4, 9, 1, 5, 5, 9])